In [ ]:
# Average departure delay of each airline per airport

# AIRLINE
# ORIGIN AIRPORT
# Total number of flight
# OPERATED RATE (non cancel rate)
# DELAY RATE (DEPARTURE) --> exclude cancelled flight
# AVERAGE DELAY MINUTES --> exclude non-delay flight

In [27]:
import pandas as pd
import numpy as np

In [28]:
df_flights = pd.read_csv('flights.csv')

/var/folders/sp/v99xtfrs4p35rhqv158v85h80000gn/T/ipykernel_1360/772266734.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_flights = pd.read_csv('flights.csv')


In [29]:
# Count number of flights by airline and origin airport
# Store results as output_table
output_table = df_flights.groupby(['AIRLINE','ORIGIN_AIRPORT']).agg(
    NUMBER_OF_FLIGHTS = ('FLIGHT_NUMBER', 'count')
).reset_index()

In [30]:
output_table.sample(5)

,AIRLINE,ORIGIN_AIRPORT,NUMBER_OF_FLIGHTS
2488,OO,12451,3
1749,HA,14747,59
1477,EV,GRK,2593
226,AA,LIT,10
2723,OO,ORF,483


In [31]:
# Count non-cancelled flights
# Store results as new_column
new_column = df_flights[df_flights['CANCELLED'] == 0].groupby(['AIRLINE','ORIGIN_AIRPORT']).agg(
    NUMBER_OF_NON_CANCELLED_FLIGHTS = ('FLIGHT_NUMBER', 'count')
).reset_index()

In [32]:
# Merge new_column to output_table
output_table = pd.merge(output_table, new_column, how = 'left', on = ['AIRLINE', 'ORIGIN_AIRPORT'])

In [33]:
output_table.sample(5)

,AIRLINE,ORIGIN_AIRPORT,NUMBER_OF_FLIGHTS,NUMBER_OF_NON_CANCELLED_FLIGHTS
126,AA,12758,5,5.0
3057,US,CMH,500,493.0
3010,UA,OGG,2151,2113.0
2370,OO,14262,389,389.0
3206,WN,11618,486,483.0


In [34]:
# Calculate operated rate
output_table['OPERATED_RATE'] = output_table['NUMBER_OF_NON_CANCELLED_FLIGHTS'] / output_table['NUMBER_OF_FLIGHTS']

In [36]:
output_table.sample(5)

,AIRLINE,ORIGIN_AIRPORT,NUMBER_OF_FLIGHTS,NUMBER_OF_NON_CANCELLED_FLIGHTS,OPERATED_RATE
3167,VX,LAS,3766,3732.0,0.990972
2064,MQ,GSP,489,465.0,0.950920
399,AS,ABQ,334,334.0,1.000000
2577,OO,AMA,458,452.0,0.986900
3134,VX,13930,137,137.0,1.000000


In [37]:
# Count delayed and non-cancelled flights
# Sum total delay minutes
# Store results as new_column
new_column = df_flights.query(
    "CANCELLED != 1 and DEPARTURE_DELAY > 0").groupby(
    ['AIRLINE','ORIGIN_AIRPORT']).agg(
    NUMBER_OF_DELAYED_DEPARTURE = ('FLIGHT_NUMBER', 'count'),
    TOTAL_DELAY_MINTUES = ('DEPARTURE_DELAY', 'sum')
).reset_index()

In [47]:
new_column.sample(5)

,AIRLINE,ORIGIN_AIRPORT,NUMBER_OF_DELAYED_DEPARTURE,TOTAL_DELAY_MINTUES
2016,NK,DTW,3383,140121.0
1466,F9,11995,5,213.0
1775,MQ,11982,3,80.0
677,DL,12451,96,1995.0
1387,EV,MFE,242,9557.0


In [39]:
# Merge new_column to output_table
output_table = pd.merge(output_table, new_column, how = 'left', on = ['AIRLINE', 'ORIGIN_AIRPORT'])

In [40]:
output_table.sample(5)

,AIRLINE,ORIGIN_AIRPORT,NUMBER_OF_FLIGHTS,NUMBER_OF_NON_CANCELLED_FLIGHTS,OPERATED_RATE,NUMBER_OF_DELAYED_DEPARTURE,TOTAL_DELAY_MINTUES
2372,OO,14487,58,58.0,1.000000,15.0,1181.0
2967,UA,CLE,6557,6465.0,0.985969,2369.0,79683.0
3228,WN,13244,254,254.0,1.000000,76.0,1816.0
1001,DL,IAD,2364,2355.0,0.996193,685.0,24521.0
39,AA,12892,3093,3083.0,0.996767,862.0,20523.0


In [42]:
# Calculate delay rate (departure)
output_table['DELAY_RATE_DEPARTURE'] = output_table['NUMBER_OF_DELAYED_DEPARTURE'] / output_table['NUMBER_OF_NON_CANCELLED_FLIGHTS']

In [43]:
# Calculate average delay minutes
output_table['AVERAGE_DELAY_MINUTES'] = output_table['TOTAL_DELAY_MINTUES'] / output_table['NUMBER_OF_DELAYED_DEPARTURE']

In [50]:
output_table.sample(5)

,AIRLINE,ORIGIN_AIRPORT,NUMBER_OF_FLIGHTS,NUMBER_OF_NON_CANCELLED_FLIGHTS,OPERATED_RATE,NUMBER_OF_DELAYED_DEPARTURE,TOTAL_DELAY_MINTUES,DELAY_RATE_DEPARTURE,AVERAGE_DELAY_MINUTES
218,AA,JAX,2360,2332.0,0.988136,685.0,29764.0,0.293739,43.451095
2942,UA,14908,29,29.0,1.000000,6.0,198.0,0.206897,33.000000
3048,US,BDL,833,811.0,0.973589,224.0,4576.0,0.276202,20.428571
1276,EV,10990,2,2.0,1.000000,1.0,7.0,0.500000,7.000000
815,DL,10980,4,4.0,1.000000,NaN,NaN,NaN,NaN


In [52]:
# Drop columns used in intermediate steps
output_without_intermediate = output_table.drop(columns = ['NUMBER_OF_NON_CANCELLED_FLIGHTS', 'NUMBER_OF_DELAYED_DEPARTURE', 'TOTAL_DELAY_MINTUES'])

In [53]:
output_without_intermediate.sample(10)

,AIRLINE,ORIGIN_AIRPORT,NUMBER_OF_FLIGHTS,OPERATED_RATE,DELAY_RATE_DEPARTURE,AVERAGE_DELAY_MINUTES
3076,US,LAX,3688,0.992679,0.322589,23.335309
2296,OO,11638,559,0.996422,0.168761,54.340426
3096,US,PHL,20501,0.975416,0.389658,31.906186
339,AS,10170,2,1.000000,NaN,NaN
2784,UA,10299,54,1.000000,0.388889,18.000000
2126,MQ,STL,1320,0.920455,0.425514,44.056093
661,DL,10599,281,1.000000,0.224199,28.158730
738,DL,13485,244,1.000000,0.163934,47.075000
736,DL,13360,110,1.000000,0.254545,29.785714
482,B6,11433,113,0.991150,0.250000,34.357143
